In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist


(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(-1, 28, 28, 1) / 255.0
x_test = x_test.reshape(-1, 28, 28, 1) / 255.0

def generate_random_number(number,x_data,y_data):
    num_samples = number
    data_numb=np.zeros((num_samples, 28, 100, 1), dtype=np.float32)
    data_labe=np.zeros((num_samples, 5), dtype=np.int_)
    for i in range(num_samples):
        digit1 = np.random.randint(0, 10)
        digit2 = np.random.randint(0, 10)
        digit3 = np.random.randint(0, 10)
        digit4 = np.random.randint(0, 10)
        digit5 = np.random.randint(0, 10)


        positions = np.random.choice([0, 1, 2], size=5)

        canvas = np.zeros((28, 100, 1), dtype=np.float32)

        lent_front = 0
        for n, position in enumerate(positions):
            digit = [digit1, digit2, digit3, digit4, digit5][n]
            id=np.random.choice(y_data[y_data == digit])
            if n==0:
                lent_front = 28*n
                x_data_front = np.zeros((28, lent_front, 1), dtype=np.float32)
                x_data_after = np.zeros((28, 72-lent_front, 1), dtype=np.float32)
            else:
                numb=np.random.randint(8,18)
                lent_front = lent_front+numb
                x_data_front = np.zeros((28, lent_front, 1), dtype=np.float32)
                x_data_after = np.zeros((28, 72-lent_front, 1), dtype=np.float32)
            canvas_add = np.concatenate([x_data_front, x_data[id]], 1)
            canvas_add = np.concatenate([canvas_add, x_data_after],  1)
            canvas += canvas_add 
            data_labe[i][n]=y_data[id]
        data_numb[i]=canvas 
    idx = list(range(len(data_numb)))
    np.random.shuffle(idx)
    data_numb = data_numb[idx]
    data_labe = data_labe[idx]
    print('end')
    return data_numb, data_labe


In [2]:
x_data_random , y_data_random = generate_random_number(80000,x_train, y_train)

end


In [3]:
x_train_ran , y_train_random = x_data_random[:60000] , y_data_random[:60000] 
x_test_ran , y_test_random = x_data_random[60000:70000] , y_data_random[60000:70000] 
x_verify_ran , y_verify_random = x_data_random[70000:] , y_data_random[70000:]  

In [74]:
from PIL import Image
from IPython.display import display

image_data = (np.squeeze(x_train_ran[0]) * 255).astype('uint8')
image = Image.fromarray(image_data, 'L')

title = f"Label:{y_train_random[0]}"
print(title)
display(image)

Label:[2 1 0 1 0]


In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Reshape, Dense, LSTM, Bidirectional
from tensorflow.keras.models import Model
from tensorflow import keras
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau 

input_shape = (28, 100, 1)

#CNN
input_img = Input(shape=input_shape, name="image", dtype="float32")
x = Conv2D(32, (3, 3), activation='tanh', kernel_regularizer=l2(0.4))(input_img)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='tanh', kernel_regularizer=l2(0.4))(x)
x = MaxPooling2D((2, 2))(x)
x = Reshape(target_shape=(5, 1472))(x)

#RNN
x = Bidirectional(LSTM(128, return_sequences=True, dropout=0.4))(x)
x = Bidirectional(LSTM(64, return_sequences=True, dropout=0.4))(x)

output = Dense(10, activation="softmax", kernel_regularizer=l2(0.4))(x) 

model = Model(inputs=input_img, outputs=output)

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])#

reduce_lr = ReduceLROnPlateau(patience =3, factor = 0.01, weightdecay = 0.05, verbose = 1) 
early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", 
                                               patience=10, restore_best_weights=True)
#train model
history = model.fit(x_train_ran, y_train_random, epochs=40, verbose=1, batch_size=200, 
          validation_data=(x_verify_ran, y_verify_random), callbacks=[early_stopping, reduce_lr ])

Epoch 1/40
300/300 [==============================] - 82s 272ms/step - loss: 4.7274 - accuracy: 0.3474 - val_loss: 1.7945 - val_accuracy: 0.3887
Epoch 2/40
300/300 [==============================] - 88s 293ms/step - loss: 1.7372 - accuracy: 0.4106 - val_loss: 1.6386 - val_accuracy: 0.4605
Epoch 3/40
300/300 [==============================] - 88s 293ms/step - loss: 1.5742 - accuracy: 0.4991 - val_loss: 1.4549 - val_accuracy: 0.5635
Epoch 4/40
300/300 [==============================] - 84s 278ms/step - loss: 1.4134 - accuracy: 0.5996 - val_loss: 1.3304 - val_accuracy: 0.6367
Epoch 5/40
300/300 [==============================] - 86s 286ms/step - loss: 1.2910 - accuracy: 0.6703 - val_loss: 1.1957 - val_accuracy: 0.7265
Epoch 6/40
300/300 [==============================] - 84s 279ms/step - loss: 1.1967 - accuracy: 0.7255 - val_loss: 1.1133 - val_accuracy: 0.7811
Epoch 7/40
300/300 [==============================] - 84s 281ms/step - loss: 1.1248 - accuracy: 0.7564 - val_loss: 1.0530 - val_ac

In [6]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2, subplot_titles=("Model accuracy", "Model loss"))

# Add the first subplot for accuracy
fig.add_trace(go.Scatter(x=list(range(1, len(history.history['accuracy']) + 1)),
                         y=history.history['accuracy'], mode='lines',
                         name='Train', line=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=list(range(1, len(history.history['val_accuracy']) + 1)),
                         y=history.history['val_accuracy'], mode='lines',
                         name='Validation', line=dict(color='orange')), row=1, col=1)

# Add the second subplot for loss
fig.add_trace(go.Scatter(x=list(range(1, len(history.history['loss']) + 1)),
                         y=history.history['loss'], mode='lines',
                         name='Train', line=dict(color='blue'), showlegend=False), row=1, col=2)
fig.add_trace(go.Scatter(x=list(range(1, len(history.history['val_loss']) + 1)),
                         y=history.history['val_loss'], mode='lines',
                         name='Validation', line=dict(color='orange'), showlegend=False), row=1, col=2)

fig.update_layout(title="Model Training History",
                  xaxis_title="Epoch",
                  yaxis_title="Value",
                  legend=dict(x=0.85, y=1))

fig.show()

In [7]:
# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(x_test_ran, y_test_random)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

313/313 [==============================] - 6s 18ms/step - loss: 0.6671 - accuracy: 0.9974
Test Accuracy: 99.74%


In [75]:
from PIL import Image
from IPython.display import display

image_data = (np.squeeze(x_test_ran[0]) * 255).astype('uint8')
image = Image.fromarray(image_data, 'L')

title = f"Label:{y_test_random[0]}; Predicted:{y_pred[0]}"
print(title)
display(image)

Label:[5 3 9 4 3]; Predicted:[5 3 9 4 3]


In [70]:
from PIL import Image, ImageDraw, ImageFont
import numpy as np

id = list(range(len(x_test)))
#打乱顺序
np.random.shuffle(id)
id_exam=id[:100]
print(len(id_exam))
for h in id_exam:
    image_data = (np.squeeze(x_test_ran[h]) * 255).astype('uint8')
    image = Image.fromarray(image_data, 'L')
    draw = ImageDraw.Draw(image)
    font = ImageFont.load_default() 
    title=f"Lab_{y_test_random[h]}-Pre_{y_pred[h]}"
    name=title.replace(" ", "")
    name=name.replace("[", "")
    name=name.replace("]", "")
    file_name = "E:/pythonfile/images/"+name+".png"
    image.save(file_name)

print("Images saved successfully.")

100
Images saved successfully.
